In [1]:
from anthropic import Anthropic

client = Anthropic(api_key="sk-xxx<your claude key>")
MODEL_NAME = "claude-3-5-sonnet-20240620"

In [2]:
import re

def calculate(expression):
    # Remove any non-digit or non-operator characters from the expression
    expression = re.sub(r'[^0-9+\-*/().]', '', expression)
    
    try:
        # Evaluate the expression using the built-in eval() function
        result = eval(expression)
        return str(result)
    except (SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError):
        return "Error: Invalid expression"


from utils import functions

tools = [f["function"] for f in functions]
print(tools)
for t in tools:
    t["input_schema"] = t["parameters"]
    t.pop("parameters")

[{'name': 'list_files', 'description': 'List all files in a directory', 'parameters': {'type': 'object', 'properties': {'directory': {'type': 'string', 'description': 'the directory to list files from'}}, 'required': ['directory']}}, {'description': 'Create a 3D model of an object with specified dimensions', 'name': 'create_3d_model', 'parameters': {'properties': {'object_name': {'description': 'Name of the object to be modeled', 'type': 'string'}, 'dimensions': {'description': 'Dimensions of the 3D object (length, width, height)', 'type': 'object', 'properties': {'length': {'type': 'number'}, 'width': {'type': 'number'}, 'height': {'type': 'number'}}, 'required': ['length', 'width', 'height']}}, 'required': ['object_name', 'dimensions'], 'type': 'object'}}, {'description': 'Get the latest insurance premium from a list of premiums.', 'name': 'latest_insurance_premium', 'parameters': {'properties': {'premiums': {'description': 'List of insurance premiums', 'type': 'array', 'items': {'ty

In [3]:
import json
def add(args: str):
    # args = json.loads(args)
    try:
        return str(float(args["a"]) + float(args["b"]))
    except Exception as e:
        return f"Error: e"


def sub(args: str):
    # args = json.loads(args)
    try:
        return str(float(args["a"]) - float(args["b"]))
    except Exception as e:
            return f"Error: e"


def mult(args: str):
    # args = json.loads(args)
    try:
        
        return str(float(args["a"]) * float(args["b"]))
    except Exception as e:
        return f"Error: e"

def div(args: str):
    # args = json.loads(args)
    try:
        
        return str(float(args["a"]) / float(args["b"]))
    except Exception as e:
        return f"Error: e"

def process_tool_call(tool_name, tool_input):
    if tool_name == "getCurrentWeather":
        content = f"temprature is 50 degree"
        return content
    elif tool_name == "calculate_distance":
        content = f"Distance is 50 miles."
        return content
    elif tool_name == "generate_password":
        content = f"Password generated: djkj2312"
        return content
    elif tool_name == "orderUmbrella":
        content = f"Order placed. the price is 10 dollars."
        return content
    elif tool_name == "list_files":
        content = "File list:\nreport.docx\ntask.txt\nnotes.txt"
        return content
    elif tool_name == "get_file_size":
        content = f"the size is 100 bytes."
        return content
    elif tool_name == "addition":
        content = add(tool_input)
        return content
    elif tool_name == "subtraction":
        content = sub(tool_input)
        return content
    elif tool_name == "multiplication":
        content = mult(tool_input)
        return content
    elif tool_name == "division":
        content = div(tool_input)
        return content
    else:
        content = "done"
        return content

def chat_with_claude(user_message):
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")
    messages = [{"role": "user", "content": user_message}]
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        messages=messages,
        tools=tools,
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {message.stop_reason}")
    print(f"Content: {message.content}")

    while message.stop_reason == "tool_use":
        messages.append({"role": "assistant", "content": message.content})
        print("==================\n")
        tool_content = []
        
        for block in message.content:
            if block.type == "tool_use":
                tool_use = block
                tool_name = tool_use.name
                tool_input = tool_use.input

                print(f"\nTool Used: {tool_name}")
                print(f"Tool Input: {tool_input}")
                print(tool_use.id)

                tool_result = process_tool_call(tool_name, tool_input)

                print(f"Tool Result: {tool_result}")

                tool_content.append({
                                    "type": "tool_result",
                                    "tool_use_id": tool_use.id,
                                    "content": tool_result,
                                })
        messages.append({
                            "role": "user",
                            "content": tool_content,
                        })
        message = client.messages.create(
            model=MODEL_NAME,
            max_tokens=4096,
            messages=messages,
            tools=tools,
        )
    else:
        response = message

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )
    print(response.content)
    print(f"\nFinal Response: {final_response}")

    return final_response

In [4]:
with open('questions.json', 'r') as file:
    data = json.load(file)
    questions = [d["user_query"] for d in data['queries']]

for q in questions:
    chat_with_claude(q)


User Message: What is the distance between San Francisco and Cupertino by driving and by air from both directions?

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(text="To answer this question, we'll need to use the calculate_distance function twice - once for driving and once for air travel. We'll calculate the distance in both directions for completeness, although typically the distance should be the same in either direction.\n\nLet's start with the calculations:", type='text'), ToolUseBlock(id='toolu_01NHM4Un7Z1vgRanfDKvWkkz', input={'origin': 'San Francisco, CA', 'destination': 'Cupertino, CA', 'mode': 'driving'}, name='calculate_distance', type='tool_use')]


Tool Used: calculate_distance
Tool Input: {'origin': 'San Francisco, CA', 'destination': 'Cupertino, CA', 'mode': 'driving'}
toolu_01NHM4Un7Z1vgRanfDKvWkkz
Tool Result: Distance is 50 miles.


Tool Used: calculate_distance
Tool Input: {'origin': 'San Francisco, CA', 'destination': 'Cupertino, CA', 'mode': 'air'}